In [1]:
# Task 1: Implementing the ROCK Algorithm
import pandas as pd
import numpy as np

# Function to categorize latitude and longitude into zones
def categorize_zone(latitude, longitude):
    num_lat_zones = 10
    num_lon_zones = 10
    lat_zone_size = (max(latitude) - min(latitude)) / num_lat_zones
    lon_zone_size = (max(longitude) - min(longitude)) / num_lon_zones
    zones = []
    for lat, lon in zip(latitude, longitude):
        lat_zone = int((lat - min(latitude)) // lat_zone_size)
        lon_zone = int((lon - min(longitude)) // lon_zone_size)
        zones.append((lat_zone, lon_zone))
    return zones

In [ ]:
# Function to handle missing values and preprocess the data
def preprocess_data(track_points_df):
    track_points_df.dropna(inplace=True)
    track_points_df['zone'] = categorize_zone(track_points_df['latitude'], track_points_df['longitude'])
    return track_points_df

# Function to compute Jaccard similarity coefficient for zones
def jaccard_similarity(A, B):
    intersection = len(set(A) & set(B))
    union = len(set(A) | set(B))
    return intersection / union if union != 0 else 0

# Function to compute links matrix using Jaccard similarity
def compute_links(data, threshold):
    num_points = len(data)
    links_matrix = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(i+1, num_points):
            similarity = jaccard_similarity(data[i], data[j])
            if similarity >= threshold:
                links_matrix[i][j] = links_matrix[j][i] = 1
    return links_matrix

In [ ]:
# ROCK Algorithm Implementation
def ROCK(data, threshold):
    links_matrix = compute_links(data, threshold)
    num_points = len(data)
    # print("len is ",num_points)
    visited = [False] * num_points
    clusters = []

    for start_node in range(num_points):
        # print(start_node)
        if not visited[start_node]:
            cluster = []
            stack = [start_node]

            while stack:
                node = stack.pop()
                if not visited[node]:
                    cluster.append(node)
                    visited[node] = True
                    for neighbor in range(num_points):
                        if links_matrix[node][neighbor] == 1 and not visited[neighbor]:
                            stack.append(neighbor)

            clusters.append(cluster)

    return clusters

# Read the data
track_points_df = pd.read_csv("go_track_trackspoints.csv")

# Preprocess the data
track_points_df = preprocess_data(track_points_df)

In [ ]:
# Extract zone data and convert it into the required format
zone_data = track_points_df['zone'].tolist()

# Implement the ROCK algorithm
threshold_rock = 0.5  # Example threshold value for ROCK
clusters_rock = ROCK(zone_data, threshold_rock)

In [ ]:
# Output the resulting clusters from ROCK
print("ROCK Clusters:")
for i, cluster in enumerate(clusters_rock):
    print(f"Cluster {i+1}: {cluster}")

In [2]:
# Task 2: Implementing the Chameleon Algorithm
import pandas as pd
import numpy as np
import networkx as nx
from scipy.spatial.distance import euclidean
from sklearn.neighbors import NearestNeighbors

# Function to categorize latitude and longitude into zones
def categorize_zone(latitude, longitude):
    num_lat_zones = 10
    num_lon_zones = 10
    lat_zone_size = (max(latitude) - min(latitude)) / num_lat_zones
    lon_zone_size = (max(longitude) - min(longitude)) / num_lon_zones
    zones = []
    for lat, lon in zip(latitude, longitude):
        lat_zone = int((lat - min(latitude)) // lat_zone_size)
        lon_zone = int((lon - min(longitude)) // lon_zone_size)
        zones.append((lat_zone, lon_zone))
    return zones

# Function to handle missing values and preprocess the data
def preprocess_data(track_points_df):
    track_points_df.dropna(inplace=True)
    track_points_df['zone'] = categorize_zone(track_points_df['latitude'], track_points_df['longitude'])
    return track_points_df

# Function for graph partitioning
def graph_partitioning(data, k):
    G = nx.Graph()
    for i, point in enumerate(data):
        print(f"Processing point {i}/{len(data)}")
        distances = [(j, euclidean(point, data[j])) for j in range(len(data)) if i != j]
        k_nearest = sorted(distances, key=lambda x: x[1])[:k]
        for neighbor, distance in k_nearest:
            G.add_edge(i, neighbor, weight=distance)
    return G

In [ ]:
# Function for agglomerative hierarchical clustering
def agglomerative_clustering(G, data):
    clusters = list(nx.connected_components(G))
    return clusters

# Function to compute the relative closeness
def relative_closeness(cluster_i, cluster_j, data):
    centroid_i = np.mean(data[cluster_i], axis=0)
    centroid_j = np.mean(data[cluster_j], axis=0)
    return 1 / (1 + euclidean(centroid_i, centroid_j))

# Function to compute the relative interconnectivity
def relative_interconnectivity(cluster_i, cluster_j, G):
    num_edges_within = G.subgraph(cluster_i).number_of_edges() + G.subgraph(cluster_j).number_of_edges()
    num_edges_between = nx.number_of_edges(G) - num_edges_within
    if num_edges_between == 0:
        return 0
    else:
        return num_edges_within / num_edges_between

In [ ]:
# Function to compute the merge cost
def merge_cost(cluster_i, cluster_j, data, G):
    closeness = relative_closeness(cluster_i, cluster_j, data)
    interconnectivity = relative_interconnectivity(cluster_i, cluster_j, G)
    return closeness * interconnectivity

# Function to merge clusters based on the merge cost
def merge_clusters(clusters, data, G):
    min_cost = float('inf')
    best_merge = None
    for i, cluster_i in enumerate(clusters):
        for j, cluster_j in enumerate(clusters[i+1:]):
            cost = merge_cost(cluster_i, cluster_j, data, G)
            if cost < min_cost:
                min_cost = cost
                best_merge = (i, j+i+1)
    if best_merge is not None:
        i, j = best_merge
        clusters[i].extend(clusters[j])
        del clusters[j]
    return clusters

In [ ]:
# Read the data
track_points_df = pd.read_csv("go_track_trackspoints.csv")

# Preprocess the data
track_points_df = preprocess_data(track_points_df)

In [ ]:
# Implement the Chameleon algorithm
k = 10  # Number of neighbors for graph partitioning
G = graph_partitioning(track_points_df['zone'], k)
clusters_chameleon = agglomerative_clustering(G, track_points_df['zone'])

In [ ]:
# Output the resulting clusters from Chameleon algorithm
print("\nChameleon Clusters:")
for i, cluster in enumerate(clusters_chameleon):
    print(f"Cluster {i+1}: {cluster}")